<a href="https://colab.research.google.com/github/jharviy/fcc_machinelearning/blob/main/book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2023-07-02 10:44:11--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   140MB/s    in 0.2s    

2023-07-02 10:44:11 (140 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
df_ratings.head()

,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0


In [5]:
df_books.head()

,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber


In [6]:
# Most books are not rated frequently and some users do not rate frequently
# To ensure statistical significance, dataset users with less than 200 ratings and books with less than 100 ratings

user_id_filtered = df_ratings["user"].value_counts().pipe(lambda x:x[x >= 200]).index

movie_id_filtered = df_ratings["isbn"].value_counts().pipe(lambda x:x[x >= 100]).index

df_filtered = df_ratings[df_ratings["user"].isin(user_id_filtered) &
                        df_ratings["isbn"].isin(movie_id_filtered)
                        ]

# Create an isbn-user matrix using pivot_table()
# Table contents are the ratings of all users for each book. If a user has not rated a book, fill with 0
df_pivot = df_filtered.pivot(index = "isbn", columns = "user", values = "rating").fillna(0)

In [7]:
# Using NearestNeighbors model and kneighbors() method to find k neighbors.
# Setting n_neighbors = 5 to find 5 similar books
# Using metric = "cosine" to find similarities between two data points. Typically used for recommendation systems
knn_model = NearestNeighbors(n_neighbors = 5, metric = "cosine")
knn_model.fit(df_pivot)

NearestNeighbors(metric='cosine')

In [8]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  recommend_num = 5
  title_id = df_books[df_books["title"] == book]["isbn"].values[0]  # get the "isbn" value or the id of the book title
  distances, indices = knn_model.kneighbors(df_pivot.loc[[title_id]], recommend_num+1) # returns the 5+1 nearest neighbors in "indices" and its "distance"
  titles = []
  for i, distance in zip(indices[0][-1:0:-1], distances[0][-1:0:-1]):
    titles.append([df_books[df_books["isbn"] == df_pivot.iloc[[i]].index[0]]["title"].values[0], distance]) # get the movie title given the indices of the df_pivot table using df_books
  recommended_books = [book, titles]
  return recommended_books

In [9]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
